# Student Loan Risk with Deep Learning

In [155]:
# Imports
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from pathlib import Path

---

## Prepare the data to be used on a neural network model

### Step 1: Read the `student-loans.csv` file into a Pandas DataFrame. Review the DataFrame, looking for columns that could eventually define your features and target variables.   

In [156]:
# Read the csv into a Pandas DataFrame
file_path = "https://static.bc-edx.com/ai/ail-v-1-0/m18/lms/datasets/student-loans.csv"
loans_df = pd.read_csv(file_path)

# Review the DataFrame
loans_df.head()

,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score,credit_ranking
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [157]:
# Review the data types associated with the columns
loans_df.dtypes

payment_history           float64
location_parameter        float64
stem_degree_score         float64
gpa_ranking               float64
alumni_success            float64
study_major_code          float64
time_to_completion        float64
finance_workshop_score    float64
cohort_ranking            float64
total_loan_score          float64
financial_aid_score       float64
credit_ranking              int64
dtype: object

In [158]:
# Check the credit_ranking value counts
loans_df["credit_ranking"].value_counts()

credit_ranking
1    855
0    744
Name: count, dtype: int64

### Step 2: Using the preprocessed data, create the features (`X`) and target (`y`) datasets. The target dataset should be defined by the preprocessed DataFrame column “credit_ranking”. The remaining columns should define the features dataset.

In [159]:
# Define the target set y using the credit_ranking column
y = loans_df["credit_ranking"]


# Display a sample of y
y[:5]

0    0
1    0
2    0
3    1
4    0
Name: credit_ranking, dtype: int64

In [160]:
# Define features set X by selecting all columns but credit_ranking
X = loans_df.copy()
X = X.drop("credit_ranking", axis=1)
# Review the features DataFrame
X.head()


,payment_history,location_parameter,stem_degree_score,gpa_ranking,alumni_success,study_major_code,time_to_completion,finance_workshop_score,cohort_ranking,total_loan_score,financial_aid_score
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4


### Step 3: Split the features and target sets into training and testing datasets.


In [161]:
# Split the preprocessed data into a training and testing dataset
# Assign the function a random_state equal to 1

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

            

### Step 4: Use scikit-learn's `StandardScaler` to scale the features data.

In [162]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the scaler to the features training dataset
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(X_train_scaled[:5])

[[-0.56794375 -1.18088385 -0.19241296 -0.17192598 -0.45737585  1.86853316
   0.7479316  -0.45120802  0.73249574 -0.22553894 -0.31915735]
 [-0.00940983  0.80065564 -0.04123135 -0.32236513 -0.0699402  -0.45384715
  -0.08558383  0.44779905 -0.03016065 -0.75844691 -1.15783431]
 [ 2.33643265 -0.84597577  1.67216021  0.05373275 -0.13451281  1.48146978
   0.80967349  1.49315611 -1.36480934  0.42579302  0.05358797]
 [ 0.21400374  1.30301776 -0.24280683 -0.17192598 -0.50042426 -0.64737885
  -0.11645477 -0.23168303 -1.36480934 -0.64002292 -0.97146166]
 [-0.00940983  1.74956187 -1.30107808 -0.24714555 -0.37127904 -1.03444223
  -0.98084114 -0.02261162  0.03339405 -0.46238693  0.51951962]]


---

## Compile and Evaluate a Model Using a Neural Network

### Step 1: Create a deep neural network by assigning the number of input features, the number of layers, and the number of neurons on each layer using Tensorflow’s Keras.

> **Hint** You can start with a two-layer deep neural network model that uses the `relu` activation function for both layers.


In [163]:
# Define the the number of inputs (features) to the model
number_input_features = len(X_train.iloc[0])

# Review the number of features
print("Number of input features:", number_input_features)


Number of input features: 11


In [164]:
# Define the number of hidden nodes for the first hidden layer
first_hidden_nodes = 8


# Define the number of hidden nodes for the second hidden layer
second_hidden_nodes = 5

# Define the number of neurons in the output layer
number_output_neurons = 1

In [165]:
# Create the Sequential model instance
from ast import mod


model = Sequential()


# Add the first hidden layer
model.add(Dense(units=first_hidden_nodes, activation='relu', input_dim=number_input_features))

# Add the second hidden layer
model.add(Dense(units=second_hidden_nodes, activation='relu'))

# Add the output layer to the model specifying the number of output neurons and activation function
model.add(Dense(units=number_output_neurons, activation='sigmoid'))


/Users/KTSnyder/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [166]:
# Display the Sequential model summary
model.summary()

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 8)              │            96 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 5)              │            45 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │             6 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 147 (588.00 B)

 Trainable params: 147 (588.00 B)

 Non-trainable params: 0 (0.00 B)

### Step 2: Compile and fit the model using the `binary_crossentropy` loss function, the `adam` optimizer, and the `accuracy` evaluation metric.


In [167]:
# Compile the Sequential model
model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    

In [168]:
# Fit the model using 50 epochs and the training data
model.fit(X_train_scaled, y_train, epochs=50, batch_size=10)


Epoch 1/50


120/120 ━━━━━━━━━━━━━━━━━━━━ 1s 775us/step - accuracy: 0.5736 - loss: 0.6962
Epoch 2/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 464us/step - accuracy: 0.6423 - loss: 0.6533
Epoch 3/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 499us/step - accuracy: 0.6988 - loss: 0.6017
Epoch 4/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 718us/step - accuracy: 0.7261 - loss: 0.5624
Epoch 5/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 549us/step - accuracy: 0.7055 - loss: 0.5594
Epoch 6/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 690us/step - accuracy: 0.7253 - loss: 0.5351
Epoch 7/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 672us/step - accuracy: 0.7314 - loss: 0.5281
Epoch 8/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 753us/step - accuracy: 0.7282 - loss: 0.5287
Epoch 9/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 803us/step - accuracy: 0.7256 - loss: 0.5276
Epoch 10/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 487us/step - accuracy: 0.7350 - loss: 0.5201
Epoch 11/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 0s 652us/step - accuracy: 0.7387 - loss: 0.5066
Epoch 12/50
120/120 ━━━━━━━━━━━━━━━━━━━━ 

### Step 3: Evaluate the model using the test data to determine the model’s loss and accuracy.


In [169]:
# Evaluate the model loss and accuracy metrics using the evaluate method and the test data
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test, verbose=2)

# Display the model loss and accuracy results
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

13/13 - 0s - 8ms/step - accuracy: 0.7550 - loss: 0.5121
Loss: 0.5121012926101685, Accuracy: 0.7549999952316284


### Step 4: Save and export your model to a keras file, and name the file `student_loans.keras`.


In [170]:
# Set the model's file path and name file 'student_loan_keras'
model_filepath = "student_loan.keras"

# Export your model to a keras file
model.save(model_filepath)


---
## Predict Loan Repayment Success by Using your Neural Network Model

### Step 1: Reload your saved model.

In [171]:
# Set the model's file path
file_path = Path("student_loan_keras.h5")

# Load the model to a new object
loaded_model = tf.keras.models.load_model(file_path)


FileNotFoundError: [Errno 2] Unable to open file (unable to open file: name = 'student_loan_keras.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

### Step 2: Make predictions on the testing data and save the predictions to a DataFrame.

In [ ]:
# Make predictions with the test data
model_predictions = loaded_model.predict(X_test_scaled)

# Display a sample of the predictions
print(model_predictions[:5])


 1/13 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
[[0.35625613]
 [0.32130164]
 [0.9146566 ]
 [0.6796349 ]
 [0.98794836]]


In [ ]:
# Save the predictions to a DataFrame and round the predictions to binary results
results = pd.DataFrame(model_predictions, columns=["predictions"])
results["actual"] = y_test.values
results["predictions"] = results["predictions"].apply(lambda x: round(x, 0)).astype(int)
results.head()

,predictions,actual
0,0,0
1,0,1
2,1,1
3,1,1
4,1,1


### Step 4: Display a classification report with the y test data and predictions

In [ ]:
# Print the classification report with the y test data and predictions
print(classification_report(results["actual"], results["predictions"]))

              precision    recall  f1-score   support

           0       0.72      0.79      0.75       188
           1       0.79      0.73      0.76       212

    accuracy                           0.76       400
   macro avg       0.76      0.76      0.76       400
weighted avg       0.76      0.76      0.76       400



---
## Discuss creating a recommendation system for student loans

Briefly answer the following questions in the space provided:

1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.

2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.

3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.

**1. Describe the data that you would need to collect to build a recommendation system to recommend student loan options for students. Explain why this data would be relevant and appropriate.**
Financial Information: This includes income, employment status, and financial dependencies. Understanding a student's financial situation helps in assessing their ability to repay loans.
Educational Background: Information about the student’s current institution, program of study, academic performance, and expected graduation date. This helps in predicting future earnings potential.
Credit History: Credit scores and existing debts, which indicate the student's reliability as a borrower.
Future Goals: Plans for further education or career paths, which might influence earning potential and loan repayment capabilities.
Loan Terms: Interest rates, repayment periods, and grace periods.
Lender Details: Reputation, reliability, customer service ratings, and past student feedback.
Financial Aid Offered: Details about subsidies, grants, or forgiveness programs associated with loans.

This data is relevant because it helps in creating personalized recommendations that consider the unique financial needs and potential of each student, ensuring that the recommended loan products are feasible and beneficial for them.

**2. Based on the data you chose to use in this recommendation system, would your model be using collaborative filtering, content-based filtering, or context-based filtering? Justify why the data you selected would be suitable for your choice of filtering method.**

Content-Based Filtering: This method recommends items similar to those a user liked in the past, based on feature similarity. In the context of student loans, the "features" would include aspects of the loans themselves (interest rates, terms) as well as student-specific data points (financial situation, academic program).
This method is appropriate because it allows the recommendation system to utilize detailed attributes of both loans and students to find the best matches. For example, students in longer academic programs might be better suited to loans with longer repayment periods or deferred payments.

**3. Describe two real-world challenges that you would take into consideration while building a recommendation system for student loans. Explain why these challenges would be of concern for a student loan recommendation system.**

Privacy and Data Security: Handling sensitive financial and personal information requires strict data protection measures to prevent breaches and misuse. This is crucial since a breach can lead to serious financial and reputational damage for users.
Bias and Fairness: There is a risk that the system could unintentionally favor certain demographics over others, based on historical data biases. For example, students from higher-income backgrounds might receive more favorable loan options compared to equally deserving students from lower-income families.
Ensuring fairness and avoiding bias in algorithmic recommendations are major concerns, as they affect the equity of access to educational opportunities. The system needs to be regularly audited for biases and adjusted to ensure fair recommendations for all students.

These challenges are significant because they directly affect the trustworthiness and effectiveness of the recommendation system, impacting its ability to serve all students equitably and securely. Addressing these issues is vital for the successful deployment and acceptance of the system among potential users.